# Сбор текстовых данных через VK API

1. Создайте приложения для работы с API VK;
2. получите свой токен для автоматизации действий в соцсети;
3. соберите не менее 2000 не пустых текстовых сообщений со стены любой открытой группы;
4. сохраните сообщения во внешний файл для дальнейшего чтения и работы.

In [18]:
import requests
import json


# noinspection PyShadowingNames
def get_text(access_token, group_id, offset: int, count):
    response_json_str = requests.get(
        f'https://api.vk.com/method/wall.get?access_token={access_token}&v=5.194&owner_id={group_id}&count={count}&offset={offset}').text
    parsed = json.loads(response_json_str)
    try:
        #return str(parsed['response']['items'][0]['text'])
        return map(lambda x: str(x['text']), parsed['response']['items'])
    except:
        print(parsed)
        raise

In [33]:
group_ids = {'itis': -35488145, 'strategium': -31092576}

In [34]:
import time

access_token = "32797754327977543279775425320fcc36332793279775452389e04a9aeaa9a29fcfab7"
group_id = group_ids['itis']

posts_with_text_to_process = 2000
filename = 'itis_posts.txt'

posts_separator = '\n––––––––––\n'
posts_per_request = 100

with open(filename, 'w', encoding="utf-8") as file:
    posts_offset = 0
    posts_with_text_found = 0
    first_post = True
    while True:
        texts = list(get_text(access_token, group_id, posts_offset, count=posts_per_request))
        if len(texts) == 0:
            print("Warning! Group doesn't have enough text posts")
            break
        time.sleep(0.1)
        posts_offset += posts_per_request
        for text in texts:

            if text != '':
                posts_with_text_found += 1

                if not first_post:
                    file.write(posts_separator)
                else:
                    first_post = False

                file.write(text)

        if posts_with_text_found >= posts_with_text_to_process:
            break